# CHURN ANALYSIS





## BUSINESS UNDERSTANDING

This is a Churn Analysis Project for a telecommunication company to determine if customers will churn.
The data for this project will be located in three different directories.The first data will be located in an SQL Server Database, the second in a GitHub repository and the final one in a One Drive.